Proiect IC : Symmetric-key Corruption Detection : When XOR-MACs Meet Combinatorial Group Testing
Lidia Istrate 342C4

XOR-GTM with t x m test matrix

In [7]:
def matrix_to_vector_power(H, R):
    t = len(H)
    m = len(H[0])
    v = len(R)
    
    result = []
    for i in range (0, v):
        result.append([])
        for j in range(0, m):
            if len(R[i]) == 1:
                result[i].append(H[R[i][0] - 1][j])
            else:
                xor_result = H[R[i][0] - 1][j] ^ H[R[i][1] - 1][j]
                if len(R[i]) > 2:
                    for k in range(2, len(R[i])):
                        xor_result = xor_result ^ H[R[i][k] - 1][j]
                        
                result[i].append(xor_result)
            
    return result
 




H = [[1, 1, 0, 0], 
     [0, 1, 1, 0],
     [0, 0, 1, 1]]

R = [[1], [2], [3], [1, 2], [2, 3], [1, 2, 3]]

result = matrix_to_vector_power(H, R)
print(result)
    

[[1, 1, 0, 0], [0, 1, 1, 0], [0, 0, 1, 1], [1, 0, 1, 0], [0, 1, 0, 1], [1, 0, 0, 1]]
